In [1]:
import sys
sys.path.append('/source/main')

In [2]:
from pathlib import Path
from itertools import chain

import pandas as pd
from tqdm import tqdm

from preprocess import preprocessor

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
def load_it():
    root = Path('/source/main/data_download/output/topics/')
    
    df = pd.DataFrame()
    for p in tqdm(root.glob('*.csv')):
        temp = pd.read_csv(str(p),usecols=['id', 'mention'])
        temp['topic'] = p.name.split('.')[0]
        df = df.append(temp)
        
    return df

In [5]:
df = load_it()

7it [00:13,  2.08s/it]


In [6]:
df.shape

(375820, 3)

In [7]:
df.sample(5)

,id,mention,topic
30009,b80a118b-3680-55d8-abb8-a27432dc905a,"__share__ {""ynm_des"":"" Mỗi bà mẹ có một hành trình nuôi con duy nhất, với muôn màu muôn sắc. Dù những hành trình ấy có khác nhau như thế nào, có một điều chắc chắn, đó là mẹ luôn l... "",""ynm_name"":"" Cộng đồng LÀM MẸ KHOA HỌC+ "",""ynm_caption"":"" youtube.com ""} __post__ Mình biết đến CLB Làm mẹ khoa học+ rất tình cờ trong buổi chiều mưa tâm trạng đang ngổn ngang giữa chăm sóc con cái và công việc thì lướt face thấy các Mom chia sẻ CLB Làm mẹ khoa học + mình cũng đăng ký tham gia. Ở đây mình học hỏi được rất nhiều kinh nghiệm nuôi dạy con từ các mẹ xinh đẹp, giỏi giang và giàu kinh nghiệm nuôi con. Mình biết thế nào là mẹ khoa học+ làm mẹ không cần quá hoàn hảo hãy cứ để các con được lớn lên một cách tự nhiên nhất, và phát triển toàn diện nhất. Mình thực sự cảm ơn CLB Làm mẹ khoa học + vì những chia sẻ rất hay và ý nghĩa như vậy! #NANOptipro4 #Lammekhoahoc+ #TheScienceOfMother+ __comment__ Oh mình thấy nhiều mẹ tham gia CLB bên này lắm nè __reply__ Clb ngoài tư vấn của chuyên gia thì còn có trao đổi kinh nghiệm của các mẹ nữa đó chị.",6732
4797,22e22b37-228e-54a3-b022-dd79844c56f5,"__share__ Các mẹ hay mua sữa NAN Optipro 4 ở đây vậy nà? Mình thì hay mua ở siêu thị hoặc mua online trên Lazada cho tiện. À, mình vừa vào Lazada đặt mua sữa tiếp thì được biết trong tháng 11, bên đây có chương trình tuần lễ mua sắm, dành nhiều quà tặng dễ thương cho bé như cẩm nang sống, ba lô, xe đạp khi mẹ mua sữa NAN Optipro 4 đó. Link mua hàng đây nè các mẹ https://goo.gl/hT78ZQ. Mẹ nhớ nhanh tay đó nghen vì chương trình chỉ diễn ra trong tháng 11 thôi\nNAN Optipro 4 có Đạm chất lượng với công thức từ Nestlé Thụy Sĩ, đạt tiêu chuẩn chất lượng CODEX (Châu Âu) và Bộ Y tế (Việt Nam) giúp con tăng cân khỏe mạnh, không béo phì nên mẹ nào cũng thích cả. Được dịp mua hàng tốt, có quà thì mẹ cứ thế mà đặt hàng thôi, hii\n\n#NestleNAN #tangcankhoemanh #khoidauvungchac __post__ __comment__ Em mới dùng Nan cho bé trộm vía bé lên cân đều, phải đặt mua kiếm xe đạp dễ thương i nhìn là mê rùi",6732
69231,da50fb0a-63f3-515b-9f4f-e1a01b3ee07f,"__share__ 👉Tin mới! tin mới đây. Mẹ nào cho con uống NAN Optipro 4 thì biết rồi đấy. Sữa NAN cực kì tốt cho sự phát triển dài lâu của bé. Nay Nan Optipro 4 còn thay áo mới nữa nhé, chiếc áo mang màu xanh mới, cao cấp, rất dễ nhận diện và đồng nhất thân thiện nữa các mẹ ơi!\n ➡️Công thức sữa thì vẫn không đổi: vẫn công thức dinh dưỡng từ Thụy Sĩ, thể hiện bằng cụm từ """"Swiss formulation"""" trên nhãn hộp giúp các mẹ dễ nhận biết được Nan Optipro 4. các mẹ biết không? Sữa NAN cung cấp Đạm Chất Lượng cho trẻ tăng cân khỏe mạnh, tăng cường sức đề kháng, tiêu hóa khỏe nữa ạ. \n 😘Sữa được nhập khẩu chính hãng bởi Nestlé Việt Nam nên chất lượng thì các mẹ có thể yên tâm tuyệt đối nhé! Các mẹ có dùng sữa này rồi, thấy sao? Chia sẻ với chị em tí kinh nghiệm nhé!\n#NANOptipro4 #Tangcankhoemanh #Khoidauvungchac __post__ __comment__ Chất lượng sữa và giá có đổi không bạn? __reply__ Sữa được nhập khẩu chính hãng bởi Nestlé Việt Nam nên chất lượng và gái sữa luôn luôn tốt nhất, phù hợp với lựa chọn của các mẹ",6732
57690,4e701a1a-c7fc-526f-92ee-c1dfb63dac46,"__share__ {""ynm_des"":"" Cha mẹ thường ít để ý là giấc ngủ của trẻ cũng là yếu tổ ảnh hưởng nhiều đến sự phát triển chiều cao. Những cách kết hợp dinh dưỡng và giấc ngủ để bé cao lớn hơn? Dinh dưỡng ban ngày sẽ ảnh hưởng như thế nào đến giấc ngủ ban đêm của trẻ nhỏ? \n \n Tất cả những thắc mắc đó sẽ được các chuyên gia và bác sĩ giải đáp trong buổi livestream tư vấn trực tuyến dành cho trẻ từ 2-6 tuổi vào thứ 7, 17/03/2018 từ 13h30 - 15h. \n \n Cả nhà nhớ đón xem livestream và gửi các câu hỏi ngay bây giờ trong phần comment nhé! "",""ynm_story"":"" Mắt Cười shared Nestlé Baby Vietnam's post. "",""ynm_name"":"" Nestlé Baby Vietnam ""} __post__ CHƯƠNG TRÌNH TƯ VẤN KIẾN THỨC DINH DƯỠNG & TÂM LÝ GIÚP CON NGỦ NGON\n👨‍⚕️👨‍⚕️👨‍⚕️ Các mẹ thông thái ơi. Con các mẹ ngủ bao nhiêu tiếng 1 ngày thế? Có mẹ nào để ý đến việc

In [8]:
SHARE = '__share__'
POST = '__post__'
COMMENT = '__comment__'
REPLY = '__reply__'

In [9]:
def mention2QA(mention):
    list_QA = []
    if POST in mention and COMMENT in mention:
        post_last_idx = mention.find(COMMENT)
        comment_last_idx = mention.find(REPLY)
        Q = mention[:post_last_idx]
        A = mention[post_last_idx+len(COMMENT):comment_last_idx]
        list_QA.append((Q, A))
    if COMMENT in mention and REPLY in mention:
        comment_start_idx = mention.find(COMMENT)
        comment_start_idx = 0 if comment_start_idx == -1 else comment_start_idx
        comment_last_idx = mention.find(REPLY)
        
        Q = mention[comment_start_idx:comment_last_idx]
        A = mention[comment_last_idx+len(REPLY):-1]
        list_QA.append((Q, A))
    return list_QA

In [10]:
mention2QA('__share__ Nói đến vấn đề chọn sữa cho con thì mẹ nào cũng luôn quan tâm nhất về dinh dưỡng, chất lượng cũng như thương hiệu sản phẩm.....\nEm thì chọn sữa NAN Optipro 4. Hiện nay sữa NAN Optipro 4 đã đổi sang bao bì màu xanh, nhưng chất lượng vẫn không hề thay đổi đâu ạ. Trên bao bì mới, các mẹ sẽ thấy dòng chữ "Swiss formulationtio". Sữa có chứa đạm chất lượng với công thức dinh dưỡng từ Nestle Thụy Sĩ. Em thấy Kathy uống sữa NAN Optipro 4 bé hấp thụ tốt, tăng cân khỏe mạnh, còn tăng cường sức đề kháng, và tiêu hóa tốt nữa ạ. Ngoài ra sữa được nhập khẩu chính hãng bởi Nestlé Việt Nam nên em càng yên tâm hơn. Mẹ nào còn đang lăn tăn chưa biết nên chọn dòng sữa nào cho bé thì hãy dùng thử sữa NAN Optipro 4 như Kathy nhà em nhé! Mình cùng nhau chia sẻ ý kiến và kinh nghiệm dùng sữa NAN Optipro 4 cho con cùng với em nhé! \n#NANOptipro4 #Tangcankhoemanh #Khoidauvungchac __post__ __comment__ chị thấy bé uống có hợp và phát triển tốt với lên cân khỏe mạnh k chị Dương Thị Thủy Bình Hà __reply__ Chị thấy Nan thay đổi diện mạo mới thế nào ạ?')

[('__share__ Nói đến vấn đề chọn sữa cho con thì mẹ nào cũng luôn quan tâm nhất về dinh dưỡng, chất lượng cũng như thương hiệu sản phẩm.....\nEm thì chọn sữa NAN Optipro 4. Hiện nay sữa NAN Optipro 4 đã đổi sang bao bì màu xanh, nhưng chất lượng vẫn không hề thay đổi đâu ạ. Trên bao bì mới, các mẹ sẽ thấy dòng chữ "Swiss formulationtio". Sữa có chứa đạm chất lượng với công thức dinh dưỡng từ Nestle Thụy Sĩ. Em thấy Kathy uống sữa NAN Optipro 4 bé hấp thụ tốt, tăng cân khỏe mạnh, còn tăng cường sức đề kháng, và tiêu hóa tốt nữa ạ. Ngoài ra sữa được nhập khẩu chính hãng bởi Nestlé Việt Nam nên em càng yên tâm hơn. Mẹ nào còn đang lăn tăn chưa biết nên chọn dòng sữa nào cho bé thì hãy dùng thử sữa NAN Optipro 4 như Kathy nhà em nhé! Mình cùng nhau chia sẻ ý kiến và kinh nghiệm dùng sữa NAN Optipro 4 cho con cùng với em nhé! \n#NANOptipro4 #Tangcankhoemanh #Khoidauvungchac __post__ ',
  ' chị thấy bé uống có hợp và phát triển tốt với lên cân khỏe mạnh k chị Dương Thị Thủy Bình Hà '),
 ('__

In [11]:
list_QA = list(df['mention'].map(mention2QA))
list_QA = list(chain(*list_QA))

In [12]:
len(list_QA)

468790

In [13]:
df_QA = pd.DataFrame(list_QA, columns=['q', 'a'])

In [14]:
df_QA.head()

,q,a
0,"__share__ Các mon cho em hỏi, em định cho gà nhà em tập ăn thêm sữa công thức mà không biết cho ăn sữa gì. Lúc mới sinh chưa có sữa em đã cho dùng friso nhưng con không ăn mấy. 1m nằm viện vì viêm phổi bố gà mua similac mà gà cũng không ăn mấy. Giờ ăn sữa gì tốt nhỉ các mon __post__",fiso nga hoặc nan nga mn h
1,__share__ gà nhà e dùng sữa frisolac mà 2 hôm nay đi ngoài phân xanh thì có phải đổi sữa k các mom? __post__,Dùng sữa ngoài là vậy á mẹ. Mình lâu lâu cug cho con ug friso cug có ít phân xanh. Ks đâ
2,"__share__ em đổi sữa morinaga cho be ga 4,5m vì friso lên kí tốt nhưng bé táo quá, đến nay đừợc 4 ngày dùng sữa và là ngày t3 sau khi ị đợt trước mà bé vẫn chưa ị, mn nào dùng mori cho gà ko ạ, e tưởng mori ị tốt chứ sao vẫn ko ị ạ, hu hu :(( __post__",Con e uông friso cũng táo quá huhu mà trc dùng dielac gold out put rõ đẹ
3,"__share__ em đổi sữa morinaga cho be ga 4,5m vì friso lên kí tốt nhưng bé táo quá, đến nay đừợc 4 ngày dùng sữa và là ngày t3 sau khi ị đợt trước mà bé vẫn chưa ị, mn nào dùng mori cho gà ko ạ, e tưởng mori ị tốt chứ sao vẫn ko ị ạ, hu hu :(( __post__",M mới dùng đc 3 ngay sưa này. Bé đi phân đêp ngay 2 lân. Chắc mát
4,__comment__ M mới dùng đc 3 ngay sưa này. Bé đi phân đêp ngay 2 lân. Chắc mát,Bé nhà m còi quá. Đang bú mẹ ht. Còi quá sốt ruột nên mới cho bé dùng sưa này 3 ngày nay xem có cải thiên đc k. 2 thang 10 ngày có 4.7k


In [16]:
df_QA.sample(10)

,q,a
87506,"__share__ {""ynm_story"":"" Enfa A+ Smart Club published a note. ""} __post__ THỂ LỆ & ĐIỀU KIỆN “KHOE ẢNH BÉ THÔNG MINH, TỰ TIN VÀ SẺ CHIA CẢM XÚC”",Ad ơi sao em k kích vô khung ảnh được ak :((. Cả link điền thông tin nữa
317818,__comment__ Cho mình hỏi Nan có số bột ăn dặm ko,Dạ Chị vui lòng tham khảo sản phẩm bột ăn dặm của NESTLÉ qua đường link https://concung.com/search?search_query=NESTL%C3%89&submit_search=T%C3%ACm+ki%E1%BA%BFm. Cảm ơn Ch
2486,"__share__ {""ynm_des"":"" Làm mẹ là một hành trình đầy khó khăn, thăng trầm nhưng cực kì hạnh phúc. Lần đầu tiên làm mẹ với bao bỡ ngỡ, vụng về và lo lắng, buồn vui có, hạnh phúc có. Cuộc sống của mẹ và cả nhà đều xoay quanh thiên thần nhỏ từ đó xả... "",""ynm_name"":"" Khóc cười chuyện nuôi con của bố mẹ thế kỷ 21 "",""ynm_caption"":"" eva.vn ""} __post__ 😘Nuôi con là cả một quá trình không hề đơn giản tí nào các mẹ nhỉ? Nhớ lại hồi giai đoạn Bình bị “táo bón” mà đến giờ mình vẫn hãi. Hồi đó mặc dù mình cũng đã chú ý chế độ dinh dưỡng nhiều rau xanh, bổ sung chất xơ nhưng không hiểu sao Bình vẫn thường xuyên bị táo bón. Nhìn con mỗi lần đi “ị” đến xanh cả mặt mà xót lắm.😓Thế rồi mình sốt ruột quá mới dẫn bé đi khám bác sỹ dinh dưỡng thì mới hiểu được vấn đề cốt lõi đó là mình chưa chọn đúng nguồn sữa “mát” cho con. Bởi thế từ khi nghe lời tư vấn bác sỹ, mình chuyển cho bé sang dùng sữa Friso thì con không bị táo bón nữa, con ăn uống cũng ngon miệng hơn nhiều. 😋Giờ nhìn thấy con cao lớn, khỏe mạnh mình thật vui và may mắn khi tìm được đúng nguồn sữa tốt, mát cung cấp cho con mỗi ngày. Mình mới đọc bài báo này mới phát hiện nhiều mẹ cũng khóc cười chuyện nuôi con giống mình nè, hihi",Con mà bị táo bón thì chắc bé khó chịu lắm chị nh
7753,"__share__ {""ynm_des"":"" Ở trẻ nhỏ, táo bón trở thành nỗi lo của cả gia đình, nhiều trẻ em bị táo bón tạm thời có thể kéo dài một vài ngày và sau ... "",""ynm_name"":"" Top 3 sữa mát dành cho trẻ táo bón - "",""ynm_caption"":"" kidsplaza.vn ""} __post__ Hôm nay lại đọc được bài viết này, mừng quá sá luôn mọi người ạ.\nChuyện là Bảo Bảo bị nóng trong. Sau khi tham khảo ý kiến của mọi người, kết hợp với tìm hiểu thông tin trên mạng, mẹ Bảo Bảo quyết định cho con dùng Friso. Friso là dòng sữa mát, thành phần thiên nhiên, nhiều chất sơ nên giúp bé tiêu hóa tốt, giảm tình trạng táo bón. Và hiện giờ, sau 1 tháng dùng Friso, Bảo Bảo đã không còn táo bón nữa trộm vía con phát triển tốt lắm. Mẹ nào có con đang gặp tình trạng táo bón hay băn khoăn muốn đổi sữa cho con thì tham khảo dòng sữa này nhé. Link cho các mẹ tham khảo nè.\n",Sữa friso mát lắm phải không chị bé nhà e đang bị táo mà nhiều người khuyên dùng friso qu
34904,__comment__ Mình đặt mua 1 lon friso gold 5 900g trên lazada mà không nhận được quà tặng như trên là sao?,Mẹ Lê Thị Thương Thương có nhận được email từ Lazada gửi xác nhận đơn hàng thành công chưa ạ. Nếu mẹ đã đặt hàng thành công chắc chắn mẹ sẽ nhận được sản phẩm Friso đặt mua và quà tặng cho bé nhé
458683,__share__ Các mẹ có con từ 2-6 tuổi đăng ký nhận quà của Nestlé ngay nhé!! \nChỉ còn tới 31/12 này là hết hạn đăng ký rồi. Đừng bỏ lỡ!\n\nQuà tặng Nestlé NAN OPTIPRO 4 với công thức dinh dưỡng Thụy Sỹ bổ sung ĐẠM CHẤT LƯỢNG rất tốt cho bé.\n\nHướng dẫn đăng ký: \n- Link đăng ký: >>> http://bit.ly/2QORmrk (hoặc bấm vào nút Đăng Ký)\n- Làm theo hướng dẫn để Nestlé gửi quà tới tận nhà nhé!!\n\nTới 31/12 này là hết chương trình rồi. Hãy đăng ký ngay để Nestlé gửi quà cho kịp nhé!! __post__,Hồng Lê
367167,"__share__ {""ynm_des"":"" Trẻ em có thể trạng và nhu cầu dinh dưỡng khác nhau. Để nuôi con một cách khoa học, mẹ cần lắng nghe cơ thể con, thấu hiểu và cung cấp đủ các dưỡng chất phù hợp, giúp con phát huy hết tiềm năng tự nhiên của mình. "",""ynm_name"":"" Mẹ cũng là nhà khoa học "",""ynm_caption"":"" thanhnien.vn ""} __post__ Quả thực có nuôi con mới thấu hiểu, với mỗi bé đều có những sở thích khác nhau cả về cách chăm sóc cũng như ăn uống. Bởi vậy mẹ chỉ cần quan sát để hiểu cơ thể trẻ cần gì hay cách thức bổ sung dinh

In [21]:
!ls /source/main/data_download/output

topics


In [22]:
df_QA.shape

(468790, 2)

In [24]:
df_QA.drop_duplicates(inplace=True)

In [25]:
df_QA.dropna(inplace=True)

In [26]:
df_QA.shape

(412910, 2)

In [36]:
df_QA.to_csv('/source/main/data_download/output/QA.csv', index=None)

In [28]:
df_QA[['q']].to_csv('/source/main/data_download/output/q.csv', index=None, header=None)
df_QA[['a']].to_csv('/source/main/data_download/output/a.csv', index=None, header=None)

In [29]:
df_QA['q'].map(lambda x: x.find('\n')!=-1).sum()

248911

In [31]:
x = pd.read_csv('/source/main/data_download/output/QA.csv', lineterminator='\n')

In [32]:
x['q'].map(lambda x: x.find('\n')!=-1).sum()

248911

In [34]:
x['a'].isnull().sum()

1

In [39]:
df_QA.sample(10)

,q,a
192410,"__share__ Các mom có ai cho bé uống sữa này cho mình xin rewei nha. Con mình đang uống friso mình định cho bé đổi qua similac mà không biết của nó có ngọt lắm ko, có béo nhiều ko. Cảm ơn các mom nhé __post__",bé mình chịu uống.bé nào hợp lên kí nhiề
259544,"__share__ {""ynm_des"":"" Mỗi bà mẹ có một hành trình nuôi con duy nhất, với muôn màu muôn sắc. Dù những hành trình ấy có khác nhau như thế nào, có một điều chắc chắn, đó là mẹ luôn l... "",""ynm_name"":"" Cộng đồng LÀM MẸ KHOA HỌC+ "",""ynm_caption"":"" youtube.com ""} __post__ 🍀🍁🍀Lần đầu làm mẹ cái gì cũng bỡ ngỡ, chưa có kinh nghiệm nên lắm lúc vất vả ghê gớm, có vấn đề gì chỉ biết trông chờ anh “google” mà có khi thông tin không chính xác. \n🌸🌿🌸Từ hồi tham gia CLB “Làm mẹ khoa học+” của nhãn hàng Nan, được giao lưu, học hỏi, chia sẻ kiến thức với các mẹ bỉm sữa giỏi giang, giàu kinh nghiệm khác cũng như các chuyên gia dinh dưỡng, bác sĩ nhi khoa, mình vỡ ra được biết bao nhiêu điều. 😻🏵Nhờ đó mà việc chăm con nhàn nhã và nhẹ nhàng hơn rất nhiều. Mong sao CLB này ngày càng kết nạp thêm nhiều thành viên và tổ chức nhiều buổi giao lưu hơn nữa để các mẹ bỉm sữa được trau dồi kiến thức và việc nuôi con thực sự không còn là... cuộc chiến. 👌✌️👍\n #NANOptipro4 #Lammekhoahoc #TheScienceOfMother\n",:
74574,__comment__ Trời ơi bú 7 ngày 1 lần mỗi lần 120ml là ok rui mà mom còn kêu có 120ml. Cân nặng như kia cũng quá ok rui mà,"May Coco nhà minh 5m20d bdau ăn nhưng minh cho ăn xíu thui, duoi 1t chủ yếu là sữa m"
401923,"__share__ {""ynm_des"":"" Toạ đàm trực tuyến đặc biệt dành cho bố mẹ có bé yêu từ 2 - 6 tuổi chủ đề “LÀM THẾ NÀO ĐỂ TRỞ THÀNH NGƯỜI MẸ KHÉO NUÔI CON KHOẺ MẠNH CẢ VỀ THỂ CHẤT LẪN TINH THẦN?\"" với sự tham gia của ThS.BS Dương Công Minh - chuyên gia Dinh Dưỡng, BS Bùi Xuân Mạnh - chuyên gia Tâm Lý và mẹ Tô Hồng Vân (mẹ Vịt) \n Ngoài ra, trong khung giờ vàng phát sóng, 50 bộ gối đi xe sẽ được tặng đến 50 bố mẹ SHARE livestream này đến các bố mẹ khác & có câu hỏi hay nhất cho bé từ 2-6 tuổi gửi về chương trình được BTC bình chọn. Các mẹ ơi, hãy bấm SHARE ngay và đặt câu hỏi ngay nhé \n Lưu ý \n 1/ Quà tặng chỉ áp dụng cho bố mẹ có câu hỏi dành cho bé từ 2-6 tuổi \n 2/ Danh sách tặng quà sẽ được công bố sau buổi livestream và quà tăng sẽ được giao tận nhà bố mẹ trong 45 ngày sau khi công bố kết quả "",""ynm_story"":"" Đặng Thuỷ shared a live video. "",""ynm_name"":"" Nestlé Baby Vietnam ""} __post__ Trước giờ mẹ luôn quan tâm sự phát triển của Ốc. Mẹ Ốc không biết nên làm thế nào để nuôi con luôn khỏe mạnh toàn diện, tiện có buổi tọa đàm của Nestlé NAN vào tham khảo ý kiến chuyên gia ngay 🌹🌹. Mọi người cùng vào xem và hỏi để giải đáp thắc mắc nào 😍😍\n#LamMeKhoaHoc+ #NuoiDuongTronTiemNang",Ngọc Trân mom cứ comt các chuyên gia sẽ trả lời. Nan sẽ lựa chọn câu hỏi hay để tặng quà n
290172,"__share__ {""ynm_des"":"" Cha mẹ thường ít để ý là giấc ngủ của trẻ cũng là yếu tổ ảnh hưởng nhiều đến sự phát triển chiều cao. Những cách kết hợp dinh dưỡng và giấc ngủ để bé cao lớn hơn? Dinh dưỡng ban ngày sẽ ảnh hưởng như thế nào đến giấc ngủ ban đêm của trẻ nhỏ? \n \n Tất cả những thắc mắc đó sẽ được các chuyên gia và bác sĩ giải đáp trong buổi livestream tư vấn trực tuyến dành cho trẻ từ 2-6 tuổi vào thứ 7, 17/03/2018 từ 13h30 - 15h. \n \n Cả nhà nhớ đón xem livestream và gửi các câu hỏi ngay bây giờ trong phần comment nhé! "",""ynm_story"":"" Mẹ Su shared Nestlé Baby Vietnam's post. "",""ynm_name"":"" Nestlé Baby Vietnam ""} __post__ 👉😴Các mẹ thấy giấc ngủ của con có quan trọng ko nè, thường mình chỉ chú ý về vấn đề dinh dưỡng, nhiều khi cũng lơ là giấc ngủ của con ha. Thật ra giấc ngủ rất quan trọng với bé, giúp con phát triển và cao lớn hơn. Nếu con được ngủ đủ giấc, giấc ngủ chất lượng tốt thì càng phát triển tốt hơn nữa.\n🌞🌚📢🍼Tất tần tật những gì liên quan đến giấc ngủ của bé sẽ được giải đáp qua chương trình LIVESTREAM Tư vấn về vấn đề “DINH DƯỠNG & TÂM LÝ GIÚP TRẺ NGỦ NGON” do Nestle NAN tổ chức vào lúc 13h40 , ngày 17/03/2018 tại fanapge Nestle Baby Vietnam, có sự